In [136]:
import pandas as pd
import pandas as pd
import pandas as pd
from complexcgr import CGR
from pathlib import Path

# Instantiate class CGR
cgr = CGR()

def get_last_name(file_path):
    return Path(file_path).stem

def preprocessing(snp_list):
    # Step1: merge SNPs of all samples
    with open(snp_list, "r") as f:
        sample_files = f.read().splitlines()

    try:
        # Read the data from the first sample
        first_data = pd.read_csv(sample_files[0], sep=" ", header=None)
        first_data.columns = ["genomic_position", "REF", get_last_name(sample_files[0])]
        # Drop the REF column
        first_data = first_data.drop(columns=["REF"])

        # Start with first_data as the merged data
        merge_data = first_data

        # add other samples and merge them one by one
        for sample in sample_files[1:]:
            print(sample)
            sample_data = pd.read_csv(sample, sep=" ", header=None)
            column_name = get_last_name(sample)
            sample_data.columns = ["genomic_position", "REF", column_name]
            # Drop the REF column
            sample_data = sample_data.drop(columns=["REF"])
            # merge data on genomic positions
            merge_data = merge_data.merge(sample_data, on="genomic_position", how="outer")

        merge_data.set_index("genomic_position", inplace=True)
        # Tranpose the df (reverse) to have samples as rows and genomic positions as columns
        merge_data = merge_data.T

        # Move the return statement outside the loop, after all samples have been processed and merged.
        return merge_data

    except Exception as e:
        print("Error:", str(e))
        return None


# Call the preprocessing function and store the result in the 'df' variable
df = preprocessing("../data/snp.lists")
#     merge_data = merge_data.fillna("N")
#     merge_data = merge_data.replace({"A": 1, "G": 2, "C": 3, "T": 4, "N": 0})
#     merge_data = merge_data.rename(columns={"V1": "POS"})
#     tinput = merge_data.T
#     tinput.columns = tinput.iloc[0]
#     tinput = tinput.iloc[1:]
#     value = len(sample_files) // 5 * 4
#     tinput4 = tinput[tinput["Num"] < value]
#     tinput5 = tinput4.T
#     tinput5.to_csv("snp_input.csv", index=False)

df=preprocessing("../data/snp.lists")
df
#     # FCGR Encoding using kaos package
#     data = pd.read_csv("snp_input.csv")
#     data = data.replace({0: "N", 1: "A", 2: "G", 3: "C", 4: "T"})

#     for i in range(900):
#         cgr_list = cgr(data[str(i)], seq_base=["N", "A", "G", "C", "T"], res=200)
#         cgr_matrix = cgr_list["matrix"]
#         cgr_matrix.to_csv(sample_files[i], index=False)


../data/SRR15279128_output.snp
../data/SRR15323717_output.snp
../data/SRR15323728_output.snp
../data/SRR15323734_output.snp
../data/SRR4123985_output.snp
../data/SRR4243040_output.snp
../data/SRR15279128_output.snp
../data/SRR15323717_output.snp
../data/SRR15323728_output.snp
../data/SRR15323734_output.snp
../data/SRR4123985_output.snp
../data/SRR4243040_output.snp


genomic_position,27,35,41,415,480,534,646,744,759,793,...,4743026,4745884,4775401,4776429,4798063,4809867,4814136,4829078,4851928,4857143
SRR15279127_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15279128_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15323717_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15323728_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15323734_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR4123985_output,NaN,NaN,NaN,NaN,NaN,C,T,C,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR4243040_output,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,A,A,C,T,A,G,C,T


In [137]:
df

genomic_position,27,35,41,415,480,534,646,744,759,793,...,4743026,4745884,4775401,4776429,4798063,4809867,4814136,4829078,4851928,4857143
SRR15279127_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15279128_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15323717_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15323728_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR15323734_output,A,A,A,T,G,C,T,C,A,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR4123985_output,NaN,NaN,NaN,NaN,NaN,C,T,C,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SRR4243040_output,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,A,A,C,T,A,G,C,T


In [90]:
# ! pip install sklearn

In [91]:
# df.columns
# df.to_csv("../data/processed_data.csv")

In [92]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
# One hot encoding

In [ ]:
# Replicate for trial 


# Create a new dataset with 500 rows
# Replicate the data 5 times (total 500 rows)
for _ in range(6):
    replicated_data = pd.concat([df, df], ignore_index=True)


In [ ]:
replicated_data

In [62]:

# Step 2: Remove constant columns
non_constant_cols = replicated_data.columns[replicated_data.nunique() > 1]
replicated_data = replicated_data[non_constant_cols]
replicated_data.reindex()

genomic_position,21392,29594,54589,75185,77367,111856,152033,177787,179637,227085,...,4625047,4644081,4652650,4685351,4753904,4786552,4803221,4845437,4847771,4848225
0,A,A,T,G,T,A,C,G,C,T,...,T,T,A,T,C,A,A,T,A,C
1,T,C,G,A,C,G,T,A,G,G,...,C,C,T,C,A,T,T,G,T,A
2,A,A,T,G,T,A,C,G,C,T,...,T,T,A,T,C,A,A,T,A,C
3,T,C,G,A,C,G,T,A,G,G,...,C,C,T,C,A,T,T,G,T,A


In [ ]:
# Step 2: Perform one-hot encoding for all categorical columns
replicated_data_encoded = pd.get_dummies(df)
replicated_data_encoded

In [58]:
# Step 3: Scale the data
scaler = MinMaxScaler()  # or StandardScaler()
scaled_data = scaler.fit_transform(replicated_data)

# Convert the scaled numpy array back to a DataFrame with the original columns
scaled_data_df = pd.DataFrame(scaled_data, columns=non_constant_cols)
scaled_data_df

NameError: name 'MinMaxScaler' is not defined